<a href="https://colab.research.google.com/github/jesse-venson/Machine-learning/blob/main/ML_assign_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score


np.random.seed(42)
n_samples = 200

X1 = np.random.rand(n_samples, 1) * 10
X2 = X1 + np.random.randn(n_samples, 1) * 0.1
X3 = X1 * 0.5 + np.random.randn(n_samples, 1) * 0.2
X4 = 2 * X1 + np.random.randn(n_samples, 1) * 0.3
X5 = X2 + X3 + np.random.randn(n_samples, 1) * 0.1
X6 = X1 + X4 + np.random.randn(n_samples, 1) * 0.5
X7 = 0.8 * X1 + 0.5 * X2 + np.random.randn(n_samples, 1) * 0.1

X = np.hstack([X1, X2, X3, X4, X5, X6, X7])
true_w = np.array([[3], [2], [4], [5], [1], [0.5], [2.5]])
y = X @ true_w + np.random.randn(n_samples, 1) * 2
y = y.flatten()

# Normalize features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X_b = np.c_[np.ones((n_samples, 1)), X]

def ridge_cost(X, y, w, lam):
    n = len(y)
    y_pred = X.dot(w)
    mse = np.mean((y_pred - y) ** 2)
    reg = lam * np.sum(w[1:] ** 2) / (2 * n)
    return mse + reg

def ridge_gradient_descent(X, y, alpha, lam, epochs=1000):
    n, m = X.shape
    w = np.zeros(m)
    cost_history = []

    for _ in range(epochs):
        y_pred = X.dot(w)
        gradient = (1/n) * X.T.dot(y_pred - y) + (lam/n) * np.r_[0, w[1:]]
        w -= alpha * gradient

        # Gradient clipping for stability
        w = np.clip(w, -1e6, 1e6)

        cost = ridge_cost(X, y, w, lam)
        if np.isnan(cost) or np.isinf(cost):
            return None, None  # skip invalid runs
        cost_history.append(cost)

    return w, cost_history


learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lambdas = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

best_r2 = -np.inf
best_params = None
best_w = None

for alpha in learning_rates:
    for lam in lambdas:
        w, cost_hist = ridge_gradient_descent(X_b, y, alpha, lam, epochs=1000)
        if w is None:  # Skip unstable runs
            continue

        y_pred = X_b.dot(w)
        if np.any(np.isnan(y_pred)) or np.any(np.isinf(y_pred)):
            continue

        r2 = r2_score(y, y_pred)
        final_cost = cost_hist[-1]

        if r2 > best_r2:
            best_r2 = r2
            best_params = (alpha, lam, final_cost)
            best_w = w


print("✅ Best Parameters Found:")
print(f"Learning Rate (α): {best_params[0]}")
print(f"Regularization (λ): {best_params[1]}")
print(f"Minimum Cost: {best_params[2]:.6f}")
print(f"Maximum R² Score: {best_r2:.6f}")
print("\nFinal Weights:\n", best_w)


✅ Best Parameters Found:
Learning Rate (α): 0.1
Regularization (λ): 1e-15
Minimum Cost: 4.847268
Maximum R² Score: 0.998968

Final Weights:
 [112.78498729   9.94892389  10.08072035   5.30426084  14.26945915
   8.50306279  10.51448683   9.946686  ]


Question 2

In [3]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

url = "https://drive.google.com/uc?id=1qzCKF6JKKMB0p7ul_1Ly8tdmRk3vE_bG"
df = pd.read_csv(url)

print("Dataset shape:", df.shape)
print(df.head())

## (a) preprocessing

# Handle null values
df = df.dropna()

# One-hot encode categorical columns
df = pd.get_dummies(df, drop_first=True)


## (b) Separate input and output features and perform scaling
X = df.drop("Salary", axis=1)
y = df["Salary"]

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##(c) fit models
ridge_alpha = 0.5748
lasso_alpha = 0.5748

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=ridge_alpha),
    "LASSO Regression": Lasso(alpha=lasso_alpha)
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    results[name] = {"R2": r2, "MSE": mse}


results_df = pd.DataFrame(results).T
print("\nModel Performance:\n")
print(results_df)

best_model = results_df["R2"].idxmax()
print(f"\n✅ Best Model: {best_model}")


if best_model == "Ridge Regression":
    reason = "Ridge performs best because it reduces overfitting by shrinking coefficients without making them zero."
elif best_model == "LASSO Regression":
    reason = "LASSO performs best because it not only reduces overfitting but also performs feature selection by setting some coefficients to zero."
else:
    reason = "Linear Regression performs best because the dataset has little multicollinearity or noise."

print("\nExplanation:", reason)


HTTPError: HTTP Error 404: Not Found

In [5]:
# RidgeCV and LassoCV on California Housing Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Load California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ridge Regression with Cross-Validation
ridge_alphas = np.logspace(-3, 3, 7)  # from 0.001 to 1000
ridge_cv = RidgeCV(alphas=ridge_alphas, cv=5)
ridge_cv.fit(X_train, y_train)
ridge_pred = ridge_cv.predict(X_test)

# Lasso Regression with Cross-Validation
lasso_alphas = np.logspace(-3, 1, 5)  # from 0.001 to 10
lasso_cv = LassoCV(alphas=lasso_alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
lasso_pred = lasso_cv.predict(X_test)

# Evaluation
print("===== Ridge Regression =====")
print(f"Best alpha: {ridge_cv.alpha_}")
print(f"R^2 Score: {r2_score(y_test, ridge_pred):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, ridge_pred)):.3f}")

print("\n===== Lasso Regression =====")
print(f"Best alpha: {lasso_cv.alpha_}")
print(f"R^2 Score: {r2_score(y_test, lasso_pred):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, lasso_pred)):.3f}")


===== Ridge Regression =====
Best alpha: 0.001
R^2 Score: 0.576
RMSE: 0.746

===== Lasso Regression =====
Best alpha: 0.001
R^2 Score: 0.577
RMSE: 0.745
